# Import libraries and metrics

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict as dd

RESULTS_PATH = "results/16c_1000ops_3/"
CLIENTS_METRICS = "clients.json"
NODES_METRICS = "node%d.json"

PLOTS_OUT_PATH = "plots/"

with open(RESULTS_PATH + CLIENTS_METRICS, "r") as f:
    clients_metrics = list(map(lambda x: json.loads(x), f.readlines()))

nodes_metrics = {}
for i in range(1, 5):
    with open(RESULTS_PATH + NODES_METRICS % i, "r") as f:
        nodes_metrics[i] = list(map(lambda x: json.loads(x), f.readlines()))


# Results

## Correctness Tests

### Correct Blockchain Order

In [ ]:
# Check if 2f+1 nodes have the same number of blocks
num_blocks = {}
for i in range(1, 5):
    num_blocks[i] = len(list(filter(lambda x: x["metric"] == "committed_block", nodes_metrics[i])))
if len(set(num_blocks.values())) > 1:
    print("Different number of blocks in nodes")
    print(num_blocks)
else:
    print("All nodes have the same number of blocks: " + str(num_blocks[1]))

# Check if all nodes have the same blocks in same order
blocks = dd(dict)
for i in range(1, 5):
    for block in list(filter(lambda x: x["metric"] == "committed_block", nodes_metrics[i])):
        blocks[i][int(block["seq"])] = block["hash"]

for i in range(1, 5):
    for j in range(1, 5):
        for k in range(1, np.max(list(num_blocks.values())) + 1):
            try:
                if blocks[i][k] != blocks[j][k] and blocks[i][k]:
                    print("ERROR: Different blocks in nodes in seq=" + str(k))
                    print("Node " + str(i) + ": " + blocks[i][k])
                    print("Node " + str(j) + ": " + blocks[j][k])
                    pass
            except KeyError:
                continue

print("All nodes have the same blocks in same order")

## Simple Statistics

### Average Block Size

In [ ]:
avg_block_size = np.mean(list(map(lambda x: int(x["num_ops"]), filter(lambda x: x["metric"] == "committed_block", nodes_metrics[1]))))
print("Average block size: " + str(avg_block_size) + " operations")

### Average immediate reply time

In [ ]:
avg_reply = np.mean(list(map(lambda x: int(x["latency"]), filter(lambda x: x["metric"] == "operation_reply", clients_metrics))))
print("Average reply time: " + str(avg_reply) + " ms")

### Average operation execution time

In [ ]:
avg_exec = np.mean(list(map(lambda x: int(x["latency"]), filter(lambda x: x["metric"] == "operation_executed", clients_metrics))))
print("Average execution time: " + str(avg_exec) + " ms")

### Executed Operations Throughput

In [ ]:
# calculate start and end
elapsed = int(clients_metrics[-1]["time"]) - int(clients_metrics[0]["time"])

# calculate number of blocks
num_blocks = len(list(filter(lambda x: x["metric"] == "operation_executed", clients_metrics)))

# calculate throughput
throughput = num_blocks / (elapsed / 1000)

print("Throughput: " + str(throughput) + " ops/s")

### Block Throughput

In [ ]:
# calculate start and end
elapsed = int(nodes_metrics[1][-1]["time"]) - int(nodes_metrics[1][0]["time"])

# calculate number of blocks
num_blocks = len(list(filter(lambda x: x["metric"] == "committed_block", nodes_metrics[1])))

# calculate throughput
throughput = num_blocks / (elapsed / 1000)

print("Throughput: " + str(throughput) + " blocks/s")


## Plotting

### Latency Per Operation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os


RESULTS_PATH = "results/"
CLIENTS_METRICS = "clients.json"

PLOTS_OUT_PATH = "plots/"

test = "fr3"

latencies_executed_runs = []
avg_latencies_runs = []

# list of latencies for each run, average them all and then average each 100 ops
for run in os.listdir(RESULTS_PATH):
    if test not in run:
        continue
    with open(RESULTS_PATH + run + "/" + CLIENTS_METRICS, "r") as f:
        latencies_executed_runs.append(list(map(lambda x: int(x["latency"]), filter(lambda x: x["metric"] == "operation_executed", list(map(lambda x: json.loads(x), f.readlines()))))))

# for each operation in a run append average to avg_latencies_runs
# mean for each operation
max_len = np.max(list(map(lambda x: len(x), latencies_executed_runs)))
print(max_len)
for i in range(0, max_len):
    avg_latencies_runs.append(np.mean(list(map(lambda x: x[i], filter(lambda l: len(l) > i, latencies_executed_runs)))))

# mean for each 100 operations
avg_latencies_runs = list(map(lambda x: np.max(x), np.array_split(avg_latencies_runs, len(avg_latencies_runs)/10)))


In [ ]:

plt.plot(avg_latencies_runs)
plt.ylabel("Latency (ms)")
plt.xlabel("Tens of Executed Operations")
plt.savefig(PLOTS_OUT_PATH + f"latency_per_operation_{test}.pdf")
plt.show()


### Throughput-Latency

In [ ]:
def throughput_run(clients_metrics_run, metric):
    elapsed = int(clients_metrics_run[-1]["time"]) - int(clients_metrics_run[0]["time"])
    num_ops = len(list(filter(lambda x: x["metric"] == metric, clients_metrics_run)))
    return num_ops / (elapsed / 1000)

In [ ]:
def avg_latency_run(clients_metrics_run, metric):
    return np.mean(list(map(lambda x: int(x["latency"]), filter(lambda x: x["metric"] == metric, clients_metrics_run))))

In [ ]:
def block_throughput_run(node_metrics_run):
    elapsed = int(node_metrics_run[-1]["time"]) - int(node_metrics_run[0]["time"])
    num_blocks = len(list(filter(lambda x: x["metric"] == "committed_block", node_metrics_run)))
    return num_blocks / (elapsed / 1000)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import re
from collections import defaultdict as dd

RESULTS_PATH = "results/"
CLIENTS_METRICS = "clients.json"
NODES_METRICS = "node%d.json"

PLOTS_OUT_PATH = "plots/"

wanted_id = 50

throughputs_executed = dd(list)
latencies_executed = dd(list)

throughputs_reply = dd(list)
latencies_reply = dd(list)

# run format : 16c_{ops_sec}ops_{run_number}
# group by ops_sec and run_number
for run in os.listdir(RESULTS_PATH):
    run_path = RESULTS_PATH + run + "/"
    # get id with regex
    id = int(re.search(r"(\d+)c", run).group(1))
    if id != wanted_id:
        continue
    # get ops_sec wit regex
    ops_sec = int(re.search(r"(\d+)ops", run).group(1))

    with open(run_path + CLIENTS_METRICS, "r") as clients_file:
        clients_metrics = list(map(lambda x: json.loads(x), clients_file.readlines()))
        throughputs_executed[ops_sec].append(throughput_run(clients_metrics, "operation_executed"))
        latencies_executed[ops_sec].append(avg_latency_run(clients_metrics, "operation_executed"))

        throughputs_reply[ops_sec].append(throughput_run(clients_metrics, "operation_reply"))
        latencies_reply[ops_sec].append(avg_latency_run(clients_metrics, "operation_reply"))


In [ ]:
#ops_sec_list = [5, 10, 20, 40, 80, 160, 320, 480, 640, 700, 860, 1000]
ops_sec_list = [5, 10, 20, 40, 80, 160, 320, 640, 1000]

# plot throughput-latency (each throughput and latency is one point in line plot)
# 1 line for replied and 1 line for executed
plt.plot(list(map(lambda x: np.mean(throughputs_executed[x])/1000, ops_sec_list)), list(map(lambda x: np.mean(latencies_executed[x])/1000, ops_sec_list)), label="Writes", marker='o', linestyle='-')
plt.plot(list(map(lambda x: np.mean(throughputs_reply[x])/1000, ops_sec_list)), list(map(lambda x: np.mean(latencies_reply[x])/1000, ops_sec_list)), label="Reads & ACKs", marker='o', linestyle='-')
plt.xlabel("Throughput (1000 ops/s)")
plt.ylabel("Latency (s)")
plt.legend()
plt.savefig(PLOTS_OUT_PATH + "throughput_latency.pdf")
plt.show()
